# Calculating folding efficiency

In [3]:
# Importing libraries
import sys
import pathlib
from pathlib import Path
import warnings
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread
warnings.filterwarnings("ignore")
from scipy.ndimage import gaussian_filter
import ipywidgets as widgets
from IPython.display import display
import bigfish.stack as stack
from PIL import Image
import importlib
import socket
import torch
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import os
import shutil
import numpy as np
from PIL import Image


# from matplotlib.animation import FuncAnimation
# from IPython.display import HTML

# Defining directories
def find_src_directory(current_directory: Path) -> Path:
    # Loop through the parent directories
    for parent in current_directory.parents:
        potential_src = parent / 'src'
        if potential_src.is_dir():
            return potential_src
    return None

# computer name 
computer_name = socket.gethostname()
computer_user_name = computer_name.split('.')[0]

# Defining directories
current_dir = pathlib.Path().absolute()
Advanced_Microscopy_dir = find_src_directory(current_dir)
sys.path.append(str(Advanced_Microscopy_dir))
import Advanced_Microscopy as AM 
importlib.reload(AM)

import ML_SpotDetection as ML
importlib.reload(ML)
model = ML.ParticleDetectionCNN()
model_path = 'particle_detection_cnn.pth'
ML.load_model(model, model_path)


In [2]:
# import cProfile
# importlib.reload(ML)

# def profile_training():
#     # your training function or call here
#     ML.run_network(image_dir='training_crops', num_epochs=300, learning_rate=0.0000005, batch_size=256)

# cProfile.run('profile_training()')

## Loading images
____

In [2]:
path_my_folder = pathlib.Path(r"/Users/nzlab-la/Library/CloudStorage/OneDrive-TheUniversityofColoradoDenver/General - Zhao (NZ) Lab/Microscope/Luis Aguilera/Live cell imaging_Folding & Nascent chains")

# data from 20240801
#data_folder_path =path_my_folder.joinpath(r"20240801 pNZ212 and pRS001_JF646_NoDelay_ch0 folding_ch1 nascent chains.lif") 
#data_folder_path =path_my_folder.joinpath(r"20240801 pNZ222 and pRS001_JF646_NoDelay_ch0 folding_ch1 nascent chains.lif")

# data from 20240801 
data_folder_path =path_my_folder.joinpath(r"20240806 pNZ212 and pRS001_JF646_NoDelay_ch0 folding_ch1 nascent chains.lif")
#data_folder_path =path_my_folder.joinpath(r"20240806 pNZ222 and pRS001_JF646_NoDelay_ch0 folding_ch1 nascent chains.lif")

# data from 20240820
#path_data_folder = pathlib.Path(r"/Users/"+computer_user_name+r"/Library/CloudStorage/OneDrive-TheUniversityofColoradoDenver/General - Zhao (NZ) Lab/Microscope/Rhiannon/Live Cell Co-translational Folding sfGFP vs wtGFP_RMS/20240820 pNZ212 or 222 and pRS001_JF646_NoDelay_50frames_ch0 folding_ch1 nascent chains")
#data_folder_path = path_data_folder.joinpath(r"20240820 pNZ212_pRS001_JF646_NoDelay_50frames.lif")  # sfGFP
#data_folder_path = path_data_folder.joinpath(r"20240820 pNZ222_pRS001_JF646NoDelay_50frames.lif") # wtGFP


In [4]:
# import cProfile
# importlib.reload(ML)

# def profile_training():
#     # your training function or call here
#     ML.run_network(image_dir='training_crops', num_epochs=1000, learning_rate=0.0000005, batch_size=256)

# cProfile.run('profile_training()')

In [3]:
list_images, list_names, pixel_xy_um, voxel_z_um, channel_names, number_color_channels,list_time_intervals, bit_depth = AM.ReadLif(data_folder_path,show_metadata=False,save_tif=False,save_png=True,format='TZYXC').read()
number_images = len(list_images)

## Selecting image
____

In [4]:
selected_image = 4
tested_image =list_images[selected_image]
print (tested_image.shape)

(20, 7, 512, 512, 2)


In [5]:
results_name = 'results_'+data_folder_path.stem + '_cell_id_'+str(selected_image)
results_folder = current_dir.parents[0].joinpath('live_cell').joinpath('results_folding',results_name)
results_folder.mkdir(parents=True, exist_ok=True)

# read the tif files containing the masks as numpy arrays
mask_file_name = 'mask_'+data_folder_path.stem+'_image_'+str(selected_image)+'.tif'
masks = imread(str(results_folder.joinpath(mask_file_name))).astype(bool)

In [8]:
#visualizer = AM.SliderPlotting(image_TZYXC=tested_image, masks=masks,cmap='gist_yarg',sigma=0.2)
#visualizer.display()

# Particle tracking
____

In [ ]:
# Parameters
channels_cytosol = [0]
channels_nucleus = [None] 
channels_spots = [1]
diameter_nucleus = None 
pixel_xy_nm = int(pixel_xy_um*1000)
voxel_z_nm = int(voxel_z_um*1000)
list_voxels =   [voxel_z_nm , pixel_xy_nm]  # , [voxel_z_nm , pixel_xy_nm] ]
list_psfs =  [voxel_z_nm ,pixel_xy_nm]  #, [voxel_z_nm , pixel_xy_nm] ]
min_length_trajectory  = 2
memory = 0
yx_spot_size_in_px = 3
channel_for_tracking = 1
maximum_spots_cluster = 3
# calculate the threshold from the histogram
starting_threshold = AM.Utilities().calculate_threshold_from_percentage(tested_image=tested_image, masks=masks, target_percentage=20)
starting_threshold

In [ ]:
importlib.reload(AM)
threshold = AM.Utilities().calculate_threshold_for_spot_detection(tested_image,list_psfs,list_voxels,channels_spots, max_spots_for_threshold = 3000, show_plot=False)
threshold

In [ ]:
importlib.reload(AM)
tracker = AM.SingleTimePointSpotDetection(image_TZYXC=tested_image, 
                                        masks=masks,list_voxels=list_voxels, 
                                        list_psfs=list_psfs,
                                        channels_spots=channels_spots, 
                                        channels_cytosol=channels_cytosol, 
                                        channels_nucleus=channels_nucleus,
                                        yx_spot_size_in_px=yx_spot_size_in_px,starting_threshold= threshold,channel_for_tracking=channel_for_tracking)    # ,channel_for_tracking=channel_for_tracking
tracker.display()

In [ ]:
starting_threshold = tracker.get_threshold_and_time()[0]
df_tracking = tracker.get_cached_dataframes()
starting_threshold
detected_spots = len(tracker.get_cached_dataframes() )

print('Number of detected spots:', detected_spots)

In [ ]:
MINIMAL_SNR = 0.5

list_dataframes_trajectories, _ = AM.ParticleTracking (image=tested_image,channels_spots=channels_spots, masks=masks, list_voxels=list_voxels,list_psfs=list_psfs, channels_cytosol=0,channels_nucleus=None,min_length_trajectory=min_length_trajectory,threshold_for_spot_detection=starting_threshold,yx_spot_size_in_px=yx_spot_size_in_px,maximum_spots_cluster=maximum_spots_cluster).run()
df_tracking= list_dataframes_trajectories[0]
threshold_tracking = starting_threshold
filtered_image = AM.Utilities().gaussian_laplace_filter_image(tested_image,list_psfs,list_voxels)

# remove low quality tracks. those that have a SNR of less than 0.1
array_selected_field_ch1= AM.Utilities().df_trajectories_to_array(dataframe=df_tracking, selected_field='snr_ch_1', fill_value='nans')
mean_snr_ch1 = np.nanmean(array_selected_field_ch1, axis=1)
std_snr_ch1 = np.nanstd(array_selected_field_ch1, axis=1)

indices_low_quality_tracks = np.where(mean_snr_ch1 <  MINIMAL_SNR)[0]

df_tracking = df_tracking[~df_tracking['particle'].isin(indices_low_quality_tracks)]
df_tracking = df_tracking.reset_index(drop=True)
df_tracking['particle'] = df_tracking.groupby('particle').ngroup()

print('Number of tracks before filtering:', len(array_selected_field_ch1))
print('Number of tracks after filtering:', len(df_tracking['particle'].unique()))

# plot image intensity histogram
plot_name_histogram = results_folder.joinpath('pixel_histogram_in_cell.png')
masked_data = tested_image * masks[np.newaxis, np.newaxis, :, :, np.newaxis].astype(float)
list_median_intensity = AM.Plots().plot_image_pixel_intensity_distribution(image=np.mean(masked_data[:,:,:,:,:],axis=0),figsize=(8, 2),bins=100,remove_outliers=True,remove_zeros=True,save_plots=True, plot_name=plot_name_histogram ,single_color =None,list_colors=channel_names,tracking_channel = channel_for_tracking,threshold_tracking=threshold_tracking)

suptitle = 'Image: ' + data_folder_path.stem[:16]+'- '+list_names[selected_image] +' - Cell_ID: '+ str(selected_image)

plot_name_original_image = results_folder.joinpath('original_image.png')
AM.Plots().plot_images(image_ZYXC=tested_image[0], df=df_tracking, masks=masks, show_trajectories=True, suptitle=suptitle,figsize=(12, 3), show_plots=True,selected_time=0, use_maximum_projection=True, use_gaussian_filter=True,cmap='binary',min_max_percentile=[0.5,99.2],show_gird=False,save_plots=True,plot_name=plot_name_original_image)

In [15]:
# selection a time point in the image and dataframe for the plot.
save_image_for_publication = False
if save_image_for_publication:
    time_point = 15
    df_time = df_tracking[df_tracking['frame'] == time_point]
    image_to_plot = tested_image[time_point,... ]
    plot_name_detection = results_folder.joinpath('detected_spots.png')
    AM.Plots().plot_selected_cell_colors( image=image_to_plot, df=df_time, spot_type=1, min_ts_size=None, show_spots=True,use_gaussian_filter = True, image_name=plot_name_detection,microns_per_pixel=pixel_xy_um,
                                        show_legend=False,list_channel_order_to_plot=[1,0,2], max_percentile=99.0, spot_color='w', spot_mark_size=4,save_image=True)

In [ ]:
df_tracking

In [ ]:
# find spots df_tracking.is_cluster is true in df_tracking
df_tracking_is_cluster = df_tracking[df_tracking['is_cluster'] == True]
df_tracking_is_cluster

print('fraction of clusters detected: ' ,len(df_tracking_is_cluster)/len(df_tracking))

#  df_tracking_is_cluster.cluster_size  > 10
df_tracking_is_cluster_large = df_tracking_is_cluster[df_tracking_is_cluster['cluster_size'] >=2]
df_tracking_is_cluster_large[['cluster_size','particle']]


## Crops

____

In [ ]:
importlib.reload(AM)
selection_threshold_for_channel_0 = 0.01
selected_field = 'snr'  # options are: psf_sigma, snr, 'spot_int'
plot_name = None
plot_name_selected_field = results_folder.joinpath('spots_'+selected_field+'.png')
log_scale = False
array_selected_field_ch0= AM.Utilities().df_trajectories_to_array(dataframe=df_tracking, selected_field=selected_field+'_ch_0', fill_value='nans') 
array_selected_field_ch1= AM.Utilities().df_trajectories_to_array(dataframe=df_tracking, selected_field=selected_field+'_ch_1', fill_value='nans')
AM.Plots().plot_crops_properties(list_particles_arrays=[array_selected_field_ch0, array_selected_field_ch1],figsize=(15, 3),save_plots=True,plot_name=plot_name_selected_field,selection_threshold=selection_threshold_for_channel_0, label =selected_field, log_scale=log_scale,list_colors=channel_names)
mean_selected_field_ch0_all_particles = np.nanmedian(array_selected_field_ch0, axis=1)
# plot snr histogram
plot_name_snr = results_folder.joinpath('histogram_snr.png')
mean_snr = AM.Plots().plot_histograms_from_df(df_tracking, selected_field=selected_field,figsize=(8,2), plot_name=plot_name_snr, bin_count=60, save_plot=True, list_colors= channel_names,remove_outliers=True)


In [ ]:
crop_size = 9
max_crops_to_display =120
selected_time_point = None

use_filtered_image= False
if use_filtered_image:
    croparray_filtered, mean_crop_filtered, first_appearance, crop_size = AM.CropArray(image=filtered_image, df_crops=df_tracking, crop_size=crop_size, remove_outliers=False, max_percentile=99.9,selected_time_point=selected_time_point).run()
else:
    croparray_filtered, mean_crop_filtered, first_appearance, crop_size = AM.CropArray(image=tested_image, df_crops=df_tracking, crop_size=crop_size, remove_outliers=False, max_percentile=99.9,selected_time_point=selected_time_point).run()

# detect spots in mean_crop_filtered
number_crops = first_appearance.shape[0]//crop_size
flag_vector = np.zeros(number_crops, dtype=bool)
for crop_id in range(number_crops):
    flag_vector[crop_id]= AM.Utilities().is_spot_in_crop(crop_id, crop_size=crop_size, selected_color_channel=0, array_crops_YXC=mean_crop_filtered,show_plot=False)

print('Number of detected spots SNR method:', np.sum(flag_vector))

plot_name_crops_filter = results_folder.joinpath('crops.png')
AM.Plots().plot_average_crops (mean_crop_filtered, crop_size, plot_orientation='horizontal',show_particle_labels=True,save_plots=True,plot_name=plot_name_crops_filter,max_crops_to_display=max_crops_to_display,flag_vector=flag_vector)

AM.Plots().plot_average_crops (first_appearance, crop_size, plot_orientation='horizontal',show_particle_labels=True,save_plots=True,plot_name=plot_name_crops_filter,max_crops_to_display=max_crops_to_display,flag_vector=flag_vector)

# Widget to select spots

___

In [19]:
def create_crop_widget(list_crops, crop_size, save_dir):
    def display_crop(crop):
        fig, ax = plt.subplots(figsize=(3, 3))
        ax.matshow(crop, cmap='gray')
        ax.grid(False)
        plt.show()
    number_crops = len(list_crops)
    # Initialize counters for filenames
    particle_counter = 0
    no_particle_counter = 0
    current_crop_id = 0  # Start from the first crop
    # Create a folder if it doesn't exist, if it exists, remove it and create a new one
    if os.path.exists(save_dir):
        if save_dir == 'training_crops':
            shutil.rmtree(save_dir)  # Remove the entire folder and its contents
            os.makedirs(save_dir)
    else:
        os.makedirs(save_dir)
    # Stack to store previously shown crops and filenames
    crop_stack = []
    filename_stack = []
    crop_id_stack = []
    # Create buttons
    spot_present_button = widgets.Button(
        description='Spot Present',
        button_style='success',  # Green button
        tooltip='Save crop as Spot Present'
    )
    no_spot_button = widgets.Button(
        description='No Spot Present',
        button_style='danger',  # Red button
        tooltip='Save crop as No Spot Present'
    )
    back_button = widgets.Button(
        description='Back',
        button_style='warning',  # Yellow button
        tooltip='Go back to the previous crop'
    )
    skip_button = widgets.Button(
        description='Skip',
        button_style='',  # Default button style
        tooltip='Skip this crop'
    )
    button_box = widgets.HBox([spot_present_button, no_spot_button, skip_button, back_button])
    output = widgets.Output()
    filename_output = widgets.Output()
    def update_crop():
        nonlocal current_crop_id
        if current_crop_id < number_crops:  # Ensure we don't go beyond the list
            crop = list_crops[current_crop_id]
            with output:
                clear_output(wait=True)
                display_crop(crop)
        else:
            with output:
                clear_output(wait=True)
                print("No more crops to display.")
    # Save crop when "Spot Present" button is clicked
    def save_spot_present(b):
        nonlocal particle_counter, current_crop_id
        if current_crop_id < number_crops:
            crop = list_crops[current_crop_id]
            save_path = os.path.join(save_dir, f'particle_crop_{particle_counter}.png')
            particle_counter += 1
            im = Image.fromarray((crop * 255).astype(np.uint8))  # Normalize and convert to uint8
            im.save(save_path)
            # Save the current state to the stack
            crop_stack.append(crop)
            filename_stack.append(save_path)
            crop_id_stack.append(current_crop_id)
            # Display the saved filename
            with filename_output:
                clear_output(wait=True)
                print(f"Saved: {save_path}")
            # Move to the next crop after saving
            current_crop_id += 1
            update_crop()
    # Save crop when "No Spot Present" button is clicked
    def save_no_spot_present(b):
        nonlocal no_particle_counter, current_crop_id
        if current_crop_id < number_crops:
            crop = list_crops[current_crop_id]

            # Define the filename and save the crop
            save_path = os.path.join(save_dir, f'no_particle_crop_{no_particle_counter}.png')
            no_particle_counter += 1
            im = Image.fromarray((crop*255 ).astype(np.uint8))  # Normalize and convert to uint8
            im.save(save_path)
            # Save the current state to the stack
            crop_stack.append(crop)
            filename_stack.append(save_path)
            crop_id_stack.append(current_crop_id)

            # Display the saved filename
            with filename_output:
                clear_output(wait=True)
                print(f"Saved: {save_path}")
            # Move to the next crop after saving
            current_crop_id += 1
            update_crop()
    # Go back to the previous crop when "Back" button is clicked
    def go_back(b):
        nonlocal current_crop_id
        if len(crop_stack) > 0 and len(filename_stack) > 0:
            # Pop the last crop, filename, and crop_id
            crop_stack.pop()
            last_filename = filename_stack.pop()
            last_crop_id = crop_id_stack.pop()
            # Reload the last crop based on its crop ID and display it
            current_crop_id = last_crop_id
            crop = list_crops[current_crop_id]
            with output:
                clear_output(wait=True)
                display_crop(crop)
            # Show the previous filename in the output
            with filename_output:
                clear_output(wait=True)
                print(f"Reverted to: {last_filename}")
        else:
            with filename_output:
                clear_output(wait=True)
                print("No previous crop to go back to.")
    # Skip the crop when "Skip" button is clicked
    def skip_crop(b):
        nonlocal current_crop_id
        if current_crop_id < number_crops:
            current_crop_id += 1
            update_crop()
    # Attach button click events
    spot_present_button.on_click(save_spot_present)
    no_spot_button.on_click(save_no_spot_present)
    back_button.on_click(go_back)
    skip_button.on_click(skip_crop)
    # Display widgets and the initial crop
    display(button_box, output, filename_output)
    update_crop()

In [20]:
create_dataset = False
if create_dataset:
    selected_color_channel = 1
    save_dir = 'training_crops'  # Define where to save the crops
    #list_crops = AM.Utilities().normalize_crop_return_list(array_crops_YXC=mean_crop_filtered,crop_size=crop_size,selected_color_channel=0,normalize_to_255=False)
    list_crops = AM.Utilities().standardize_spot_return_list(array_crops_YXC=mean_crop_filtered,crop_size=crop_size,selected_color_channel=0)
    create_crop_widget(list_crops=list_crops, crop_size=crop_size, save_dir=save_dir)

## Model Training

____

In [51]:


load_model_from_file = True
model_name = 'particle_detection_cnn.pth'
if load_model_from_file == False:
    importlib.reload(ML)
    model, training_losses, validation_losses = ML.run_network(image_dir='training_crops', num_epochs=10000,learning_rate = 0.0000005,batch_size = 256)
    #model, training_losses, validation_losses  = ML.run_network(images_dir='training_crops', num_epochs=100, learning_rate=0.0000005, batch_size=256)
    ML.save_model(model, path=model_name)
else:
    model = ML.ParticleDetectionCNN()
    model_path = model_name
    ML.load_model(model, model_path)

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(training_losses, label='Training Loss', color='blue', marker='o', linestyle='-', linewidth=2, markersize=6)
plt.plot(validation_losses, label='Validation Loss', color='red', marker='x', linestyle='--', linewidth=2, markersize=8)
plt.title('Training and Validation Loss Over Epochs', fontsize=16)
plt.xlabel('Epochs', fontsize=14)
plt.ylabel('Loss', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()  # Adjusts plot margins to give a better layout
plt.show()

## Model Implementation
_____

In [ ]:
importlib.reload(ML)
selected_color_channel = 0
list_crops = AM.Utilities().normalize_crop_return_list(array_crops_YXC=mean_crop_filtered,crop_size=crop_size,selected_color_channel=0,normalize_to_255=True)
flag_vector = ML.predict_crops(model, list_crops,threshold=0.51)

print("Number of predicted particles ML method:", np.sum(flag_vector))


In [ ]:
AM.Plots().plot_average_crops (mean_crop_filtered, crop_size, plot_orientation='horizontal',show_particle_labels=True,save_plots=True,plot_name=plot_name_crops_filter,max_crops_to_display=120,flag_vector=flag_vector)


## Network Visualization

____

In [4]:
import torch
from torchviz import make_dot
from torch.utils.tensorboard import SummaryWriter


# Create a function to visualize the model
def visualize_model(model, input_size=(1, 1, 64, 64)):
    # Create a dummy input tensor of the same shape as your actual inputs
    dummy_input = torch.randn(*input_size)
    # Perform a forward pass to capture the computation graph
    output = model(dummy_input)
    # Visualize the computation graph using torchviz
    dot = make_dot(output, params=dict(model.named_parameters()))
    dot.render("particle_detection_cnn_computation_graph", format="png")  # Save as PNG
    print("Network visualization saved as 'particle_detection_cnn_computation_graph.png'.")

# Visualize your model's computation graph
model = ML.ParticleDetectionCNN()
visualize_model(model)

Network visualization saved as 'particle_detection_cnn_computation_graph.png'.


In [5]:
from torchsummary import summary

# Assuming your ParticleDetectionCNN class is defined
model = ML.ParticleDetectionCNN().to('cpu')  # You can use 'cuda' or 'mps' if available
summary(model, input_size=(1, 64, 64))  # For a single-channel input of size 64x64


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 64, 64]             320
         MaxPool2d-2           [-1, 32, 32, 32]               0
            Conv2d-3           [-1, 64, 32, 32]          18,496
         MaxPool2d-4           [-1, 64, 16, 16]               0
            Linear-5                  [-1, 128]       2,097,280
            Linear-6                    [-1, 1]             129
Total params: 2,116,225
Trainable params: 2,116,225
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.02
Forward/backward pass size (MB): 1.88
Params size (MB): 8.07
Estimated Total Size (MB): 9.96
----------------------------------------------------------------


In [ ]:
import torch
import torch.onnx

# Initialize the model
model =  ML.ParticleDetectionCNN().to('cpu') 

# Create a dummy input tensor with the same size as your model input (e.g., a 64x64 grayscale image)
dummy_input = torch.randn(1, 1, 64, 64)

# Export the model to ONNX format
torch.onnx.export(model, dummy_input, "particle_detection_model.onnx",
                  input_names=['input'], output_names=['output'],
                  verbose=True)

print("Model exported to particle_detection_model.onnx")